In [1]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [2]:
fred_key = 'a4459da91e23d9116f98724098d92428'
fred = Fred(api_key=fred_key)
today = date.today()


# Get Series Data
def get_fred_series_data(fred_key,series, **kwargs):
  # url
  url = "https://api.stlouisfed.org/fred/series/observations?series_id={}&api_key={}&file_type=json".format(series, fred_key)

  # response
  response = requests.get(url)
  data = response.json()
  return data


def get_fred_data(param_list, start_date, end_date):
  df = pdr.DataReader(param_list, 'fred', start_date, end_date)
  return df.reset_index()

In [13]:
code_all = pd.read_excel('C:/Users/fanjz/Desktop/python/gdp/gdp_code.xlsx', sheet_name='all')

code_annual = code_all[code_all['Frequency'] == 'Annual']
code_quarter = code_all[code_all['Frequency'] == 'Quarterly']

code_annual_list = code_annual['Code'].tolist()
code_quarter_list = code_quarter['Code'].tolist()

In [19]:
data_annual = get_fred_data(code_annual_list, start_date='1950-01-01', end_date=str(today))
data_quarter = get_fred_data(code_quarter_list, start_date='1950-01-01', end_date=str(today))

data_annual =  data_annual.rename(columns={'DATE' : 'Date'})
data_quarter =  data_quarter.rename(columns={'DATE' : 'Date'})

In [29]:
data_annual_df = data_annual.melt(id_vars='Date', var_name='Code', value_name='Values')
data_quarter_df = data_quarter.melt(id_vars='Date', var_name='Code', value_name='Values')

In [31]:
data_annual_merge = pd.merge(data_annual_df, code_annual, on='Code')
data_quarter_merge = pd.merge(data_quarter_df, code_quarter, on='Code')

In [34]:
data_quarter_merge['Year'] = data_quarter_merge['Date'].dt.year
data_quarter_merge['Quarter'] = pd.PeriodIndex(data_quarter_merge['Date'], freq='Q').quarter
data_quarter_merge['Year_quarter'] = data_quarter_merge['Year'].astype(str) + 'Q' + data_quarter_merge['Quarter'].astype(str)

In [39]:
data_annual_merge = data_annual_merge.dropna(subset=['Values'])
data_quarter_merge = data_quarter_merge.dropna(subset= ['Values'])

data_annual_merge['Date'] = data_annual_merge['Date'].dt.strftime('%Y-%m-%d')
data_quarter_merge['Date'] = data_quarter_merge['Date'].dt.strftime('%Y-%m-%d')


In [40]:
data_annual_merge.to_excel('C:/Users/fanjz/Desktop/python/gdp/quarterly_gdp_data.xlsx', index=False)
data_quarter_merge.to_excel('C:/Users/fanjz/Desktop/python/gdp/yearly_gdp_data.xlsx', index=False)